In [1]:
import pandas as pd
import datetime
from dateutil.relativedelta import relativedelta
import FinanceDataReader as fdr
import numpy as np
import re
from marcap import marcap_data
import time
import matplotlib.pyplot as plt
from pykrx import stock

# 필요 함수 정리

In [252]:
def date_str (date):
    date_temp=str("%s-%02d-%02d" %(date.year, date.month, date.day))
    return date_temp
def date_str1 (date):
    date_temp=str("%s%02d%02d" %(date.year, date.month, date.day))
    return date_temp
# 1개월 이전의 기간 동안의 수익률

# def first_date (date):
    

def Change_acc (corp_code, date, duration):
    gap=int('1') # 매수매도 판단 시점 기준으로 몇달 전의 수익을 가지고 판단하는지
    start = date_str(date-relativedelta(months=(int(duration)+gap)))
    end = date_str(date-relativedelta(months=gap,days=1))
    
    df_stock_yield=fdr.DataReader(corp_code,start,end)
    df_stock_yield['Change_acc']=(1+df_stock_yield['Change']).cumprod()
    return df_stock_yield.tail(1)['Change_acc'][0]

def output_dart (text):
    txt = 'Raw/Dart/{}.txt'.format(text)
    dart = pd.read_csv(txt, encoding = 'CP949', sep = '\t')
    dart1 = dart.loc[:,['종목코드','회사명','결산월','결산기준일','보고서종류','통화','항목명','당기']]

    word_list = []
    for i in dart1['항목명'].index:
        word = dart1['항목명'][i]
        word = re.compile('[^|가-힣]+').sub('',str(word))
        word_list.append(word)
    dart1 = dart1.drop(['항목명'],axis=1)
    dart1.rename(columns = {'당기':'2019 손익계산서'},inplace=True)
    dart1.insert(loc=(len(dart1.columns)-1),column='항목명',value=word_list)
    return dart1

# dataframe 정렬하기
def dataframe_reset(df,stock_code):
    # 첫열을 index로 수정하기
    df.index = df.iloc[:,0]
    df = df.drop(columns = df.columns[0])
    df.index.name = None
    
    # 결측 columns 삭제하기
    data_temp = df.isnull().sum().to_frame()
    nan_list = data_temp.index[data_temp[0] == len(df.index)]
    df = df.drop(columns=nan_list)
        
    # column에 사용할 하나의 tuple type행으로 만들기
    list_temp=[]
    df_columns_temp=[]
    for x in range(len(df.columns)):
        for y in range(len(df.index)):
            list_temp.append((df.columns[x][0],df.columns[x][1],df.index[y]))
    tuples = list_temp        
    index = pd.MultiIndex.from_tuples(tuples)
    # 빈 dataframe만들기
    df_temp = pd.DataFrame(index=[stock_code], columns=index)
        
    # 빈 dataframe에 값 입력하기
    # Unnamed: 이면 패스하기 / Unnamed 가 여러개 라서 error가 발생하는 듯.
    for j in range(len(df.columns)):
        for i in range(len(df.index)):
            df_temp.at[stock_code,(df.columns[j][0],df.columns[j][1],df.index[i])]=df.loc[df.index[i],(df.columns[j][0],df.columns[j][1])]
    dataframe_stock = df_temp
    return dataframe_stock

def yield_fdr(stock_list,bgn,end):
    dist_list = []
    for j in range(len(stock_list)):
        dist_list.append(1/len(stock_list))

    date1 = date_str(bgn)
    date2 = date_str(end)

    # 수익률 dataframe으로 만드는 함수 추가
    df_yield = pd.DataFrame()
    for i in range(len(stock_list)):
        df_temp = fdr.DataReader(stock_list[i],date1,date2)['Change'].to_frame()
        df_temp['Change_acc']=(1+df_temp['Change']).cumprod() * dist_list[i]
        df_temp = df_temp.drop(['Change'],axis=1)
        df_temp = df_temp.rename(columns={'Change_acc':stock_list[i]})
        df_yield = pd.concat([df_yield,df_temp],axis=1)
    df_yield.loc[date1:date2,'Change_all']=df_yield.sum(axis=1)
    return df_yield

def yield_change(stock_list,bgn,end): # 현재 사용 중
    # 수익률 자료에 없는 종목들이 걸려서 제거.
    stock_list_temp = []
    for i in stock_list:
        if str('%06d'%(int(i))) in change.columns:
            stock_list_temp.append(str('%06d'%(int(i))))
        else:
            print(str('%06d'%(int(i))))

    stock_list = stock_list_temp
    
    dist_list = []
    for j in range(len(stock_list)):
        dist_list.append(1/len(stock_list))

    date1 = date_str(bgn)
    date2 = date_str(end)

    # 수익률 dataframe으로 만드는 함수 추가
    df_yield = pd.DataFrame()
    for i in range(len(stock_list)):
        stock_code = '%06d'%(int(stock_list[i])) # 앞에 0이 사라져 다시 변환함
        df_temp = change.loc[date1:date2][stock_code].to_frame()
        df_temp.columns = ['Change']
        df_temp['Change_acc']=(1+df_temp['Change']).cumprod() * dist_list[i]
        df_temp = df_temp.drop(['Change'],axis=1)
        df_temp = df_temp.rename(columns={'Change_acc':stock_code})
        df_yield = pd.concat([df_yield,df_temp],axis=1)
    df_yield.loc[date1:date2,'Change_all']=df_yield.sum(axis=1)
    return df_yield

def price_change(list,bgn,end):
    date1 = date_str(bgn)
    date2 = date_str(end)
    df = price.loc[date1:date2, list]
    df_change = pd.DataFrame(columns = df.columns, index = df.index)
    for j in range(len(df.columns)):
        for i in range(len(df.index)-1):
            df_change.iloc[0,j] = 0
            df_change.iloc[i+1,j] = df.iloc[i+1,j] / df.iloc[i,j]-1
        df_change.loc[:,df.columns[j]] = (1+df_change[df.columns[j]]).cumprod()/len(df.columns)
    df_change.loc[:,'Change_all']=df_change.sum(axis=1)
    return df_change

def yield_fdr_simple(stock_list,bgn,end):
    dist_list = []
    for j in range(len(stock_list)):
        dist_list.append(1/len(stock_list))

    date1 = date_str(bgn)
    date2 = date_str(end)
    
    # 수익률 dataframe으로 만드는 함수 추가
    df_yield = pd.DataFrame()

    for i in stock_list:
        price1 = fdr.DataReader(i,date1,date2).iloc[0,3]
        price2 = fdr.DataReader(i,date3,date4).iloc[0,3]
        df_temp = pd.DataFrame(data=[price1,price2,price2/price1*dist_list[j]],columns=[i],index=[date1,date3,'yield'])
        df_yield = pd.concat([df_yield,df_temp],axis=1)
    df_yield.loc[:,'Change_all']=df_yield.sum(axis=1)
    return df_yield

def making_cost_list (bgn):
    date1 = date_str(bgn)
    date2 = date_str(bgn+relativedelta(days=7))
    first_date=fdr.DataReader('005930',date1,date2).index[0]
    df1 = stock.get_market_ohlcv_by_ticker(date_str1(first_date), "KOSPI")
    df2 = stock.get_market_ohlcv_by_ticker(date_str1(first_date), "KOSDAQ")
    df = pd.concat([df1,df2],axis=0)
    cost_list = df.sort_values(by='시가총액',ascending=True)
    cost_list = cost_list.loc[lambda cost_list:cost_list['거래대금']> np.int64('100000000'),:] #거래대금 1억 이상
    total_list = cost_list.index
    list_temp=[]
    p=re.compile('[0-9]*5$|[0-9]*7$|[0-9]*9$|[0-9]*K$|[0-9]*L$|[0-9]*M$|[0-9]*B$')
    for i in total_list:
        if not p.match(i):
            list_temp.append(i)
    cost_list = cost_list.loc[list_temp,:] # 우선주 제외
    cost_list = cost_list.index
    return cost_list

# 종목 List에서 원하는 항목

### 종목 data 불러오기 (by fnguid)

In [23]:
# csv 읽어서 dataframe 다시 만들기
c = pd.read_csv('fnguide_all.csv',low_memory=False)
# index 지정
c = c.set_index(c.columns[0])
c.index.name = None
# column 지정
c.columns = [c.columns.map(lambda x : x[:3]), c.iloc[0], c.iloc[1]]
c = c.iloc[2:]
c.index.name = None
# 종목명 6자리로 변경 / 시간이 오래 걸림
for i in range(len(c.index)):
    c.rename(index={c.index[i]:'%06d'%(c.index[i])}, inplace = True)

Ann                                                     ...  \
nan    2014/12                                                     ...   
nan     BPS(원) DPS(원) EPS(원)  PBR  PER  ROA  ROE 당기순이익  매출액 발행주식수  ...   
095570     NaN    NaN    NaN  NaN  NaN  NaN  NaN   NaN  NaN   NaN  ...   
006840     NaN    NaN    NaN  NaN  NaN  NaN  NaN   NaN  NaN   NaN  ...   
027410     NaN    NaN    NaN  NaN  NaN  NaN  NaN   NaN  NaN   NaN  ...   
282330     NaN    NaN    NaN  NaN  NaN  NaN  NaN   NaN  NaN   NaN  ...   
138930     NaN    NaN    NaN  NaN  NaN  NaN  NaN   NaN  NaN   NaN  ...   
...        ...    ...    ...  ...  ...  ...  ...   ...  ...   ...  ...   
024060     NaN    NaN    NaN  NaN  NaN  NaN  NaN   NaN  NaN   NaN  ...   
010240     NaN    NaN    NaN  NaN  NaN  NaN  NaN   NaN  NaN   NaN  ...   
189980     NaN    NaN    NaN  NaN  NaN  NaN  NaN   NaN  NaN   NaN  ...   
037440     NaN    NaN    NaN  NaN  NaN  NaN  NaN   NaN  NaN   NaN  ...   
238490     NaN    NaN    NaN  NaN  NaN  NaN  NaN   NaN  NaN   NaN  ...   

           Net                                                               
nan    2020/08                                                               
nan       영업이익 영업이익(발표기준) 영업이익률  유보율  자본금 자본총계 자산총계 지배주주순이익 지배주주순이익률 지배주주지분  
095570     NaN        NaN   NaN  NaN  NaN  NaN  NaN     NaN      NaN    NaN  
006840     NaN        NaN   NaN  NaN  NaN  NaN  NaN     NaN      NaN    NaN  
027410     NaN        NaN   NaN  NaN  NaN  NaN  NaN     NaN      NaN    NaN  
282330     NaN        NaN   NaN  NaN  NaN  NaN  NaN     NaN      NaN    NaN  
138930     NaN        NaN   NaN  NaN  NaN  NaN  NaN     NaN      NaN    NaN  
...        ...        ...   ...  ...  ...  ...  ...     ...      ...    ...  
024060     NaN        NaN   NaN  NaN  NaN  NaN  NaN     NaN      NaN    NaN  
010240     NaN        NaN   NaN  NaN  NaN  NaN  NaN     NaN      NaN    NaN  
189980     NaN        NaN   NaN  NaN  NaN  NaN  NaN     NaN      NaN    NaN  
037440     NaN        NaN   NaN  NaN  NaN  NaN  NaN     NaN      NaN    NaN  
238490     NaN        NaN   NaN  NaN  NaN  NaN  NaN     NaN      NaN    NaN  

[2220 rows x 1593 columns]

### 종목 data로 PBR 확인 (필요한지 확인 필요)

In [ ]:
# 2019.12에 결산이 없는 경우는 전부 제외.
df_base = pd.DataFrame()
for j in ['2015','2016','2017','2018','2019']:
    df_temp = c[('Ann','{}/12'.format(j))].loc[:,['BPS(원)','발행주식수']]
    df_temp.dropna(subset=['BPS(원)'],inplace=True) #해당기준월 결측치 있으면 제외
    df_temp = df_temp.rename(columns = {'BPS(원)':'BPS(원)_{}'.format(j),'발행주식수':'발행주식수_{}'.format(j)})
    
    bgn=datetime.datetime(int(j)+1,3,1)
    date1 = date_str(bgn)
    date2 = date_str(bgn+relativedelta(days=7))
    first_date=fdr.DataReader('005930',date1,date2).index[0]
    marcap = marcap_data(first_date).loc[:,['Code','Marcap']]
    
    print('{}년도 시작'.format(j))

    for i in df_temp.index:
        try:
            df_temp.loc[i,'시가총액_{}'.format(j)]=marcap[marcap['Code'] == i].iloc[0,1]
#             marcap[marcap['Code'] == i].iloc[0,1]
#             fdr.DataReader(i,date1,date2).iloc[0,3]
        except:
            continue
    
    print('{}년도 종료'.format(j))
    
    df_temp.dropna(subset=['시가총액_{}'.format(j)],inplace=True)
    df_temp = df_temp.astype(float) #자료형 전체 바꾸기
    df_temp.loc[:,'PBR_{}'.format(j)] = df_temp['시가총액_{}'.format(j)] / ( df_temp['BPS(원)_{}'.format(j)] * df_temp['발행주식수_{}'.format(j)] * 1000 )
    df_base = pd.concat([df_base,df_temp], axis=1)
# corp_code = df_temp.index[0]
# df_temp.loc[corp_code,'주가_2018']=marcap[marcap['Code'] == corp_code].iloc[0,1]
df_base

### 가격 data 불러오기

In [3]:
# csv 읽어서 dataframe 다시 만들기
price = pd.read_csv('price_all.csv',low_memory=False)
# index 지정
price = price.set_index(price.columns[0])
# .1 삭제
del_list=[]
for i in price.columns:
    if '.1' in i:
        del_list.append(i)
price.drop(columns = del_list, inplace=True)

### 변화율 data 불러오기

In [155]:
# csv 읽어서 dataframe 다시 만들기
change = pd.read_csv('change_all.csv',low_memory=False)
# index 지정
change = change.set_index(change.columns[0])
# .1 삭제
del_list=[]
for i in change.columns:
    if '.1' in i:
        del_list.append(i)
change.drop(columns = del_list, inplace=True)
change = change.fillna(-1) # 값이 없는 경우 상장폐지로 보고, -1을 넣어서 수익률에 고려하게 적용

### 시가총액 data 불러오기

In [134]:
from pykrx import stock
df1 = stock.get_market_ohlcv_by_ticker("20180102", "KOSPI")
df2 = stock.get_market_ohlcv_by_ticker("20180102", "KOSDAQ")
df = pd.concat([df1,df2],axis=0)

### 거래대금 소액 제외 및 우선주 제외하고 시가총액 순으로 정렬

In [139]:
cost_list = df.sort_values(by='시가총액',ascending=True)
cost_list = cost_list.loc[lambda cost_list:cost_list['거래대금']> np.int64('100000000'),:] #거래대금 1억 이상
total_list = cost_list.index
list_temp=[]
p=re.compile('[0-9]*5$|[0-9]*7$|[0-9]*9$|[0-9]*K$|[0-9]*L$|[0-9]*M$|[0-9]*B$')
for i in total_list:
    if not p.match(i):
        list_temp.append(i)
cost_list = cost_list.loc[list_temp,:] # 우선주 제외
# cost_list = cost_list.index
cost_list

,종목명,시가,고가,저가,종가,거래량,거래대금,시가총액,시총비중,상장주식수
종목코드,,,,,,,,,,
263770,신영스팩3호,2230,2280,2175,2210,146047,322788915,5757050000,0.000000,2605000
099350,하나니켈2호,2295,2435,2295,2375,93099,220437590,7799500000,0.000000,3284000
217810,엔에이치스팩7호,2025,2025,2020,2025,54300,109694365,9780750000,0.000000,4830000
223040,교보5호스팩,2000,2005,1985,2000,87187,173582300,10860000000,0.000000,5430000
051980,넥스트바이오홀딩스,830,830,810,820,155416,127585208,16799034140,0.010002,20486627
...,...,...,...,...,...,...,...,...,...,...
035420,NAVER,871000,885000,870000,885000,93587,82418272700,29171970915000,1.809570,32962679
005490,POSCO,335000,341000,332500,339000,153013,51616111000,29556337065000,1.830078,87186835
005380,현대차,150500,153000,149000,149500,731763,110423755000,32931333610500,2.039062,220276479


# 값을 통해 Score 부여

# Score를 기준으로 종목 선정

### 기준으로 정렬

### PBR 정렬

In [8]:
# 값이 없는 경우 제외
df_base_year = df_base.dropna(subset=['PBR_2015']) #결측치 제외
df_base_year = df_base_2015[df_base_2015['PBR_2015'] > 0.2] #0.2보다 작은 경우 제외(BPS가 마이너스)
df_base_year_list = df_base_2015.sort_values(by='PBR_2015', ascending=True).index
# len(df_base_2015_list)

### Low Cost 정렬
#### Marcap으로 정렬하면, Konex등 기타 불필요한 종목이 포함됨

In [129]:
making_cost_list(bgn)

Index(['053060', '005030', '114120', '214310', '009730', '058530', '056000',
       '006200', '317530', '197140', '139050', '079170', '025870', '053110',
       '032580', '154040', '037400', '065690', '013720', '227100', '085810',
       '052600', '099410', '030720', '019570', '045340', '024940', '101170',
       '019770', '045890', '066910', '064480', '051980', '204840', '003780',
       '173130', '002870', '230980', '277410', '109080', '054940', '053290',
       '059120', '010280', '290660', '021650', '060260', '131220', '115500',
       '217620'],
      dtype='object', name='종목코드')

In [50]:
bgn=datetime.datetime(2018,1,1)
end = datetime.datetime(2018,1,10)
change.loc[date_str(bgn):date_str(end)]['000660']

Date
2018-01-02    0.001307
2018-01-03    0.014360
2018-01-04   -0.007722
2018-01-05    0.028534
2018-01-08   -0.013871
2018-01-09   -0.016624
2018-01-10   -0.052016
Name: 000660, dtype: float64

In [241]:
test_list_all=[]
start = time.time()
k='2010'
bgn=datetime.datetime(int(k),1,1)
end=datetime.datetime(int(k)+1,1,1)
# 조건 (시가총액으로 종목정렬)
df_base_year_list = making_cost_list(bgn)
# 종목 구성
div = 10 # 몇개로 쪼갤지
stock_list_all=[]
for i in range(div):
    stock_list_temp = df_base_year_list[int(len(df_base_year_list)/div*i):int(len(df_base_year_list)/div*(i+1))]
    stock_list_all.append(stock_list_temp)

In [247]:
change['045400']

Date
1990-01-03   -1.0
1990-01-04   -1.0
1990-01-05   -1.0
1990-01-06   -1.0
1990-01-08   -1.0
             ... 
2020-12-01   -1.0
2020-12-02   -1.0
2020-12-03   -1.0
2020-12-04   -1.0
2020-12-07   -1.0
Name: 045400, Length: 8057, dtype: float64

In [253]:
# change.loc[date1:date2]['033300']
# len(df_base_year_list)
# '033300' in change.columns
# type(change.columns[0])
stock_list_all[0]
yield_change(stock_list_all[0],bgn,end)

,045400,064060,001600,033280,054020,054150,004820,038690,037110,024870,...,003310,053040,053700,054010,015170,035200,060240,015260,036620,Change_all
Date,,,,,,,,,,,,,,,,,,,,,
2010-01-04,0.009217,0.007595,0.007917,0.008151,0.007398,0.008468,0.007656,0.007825,0.008141,0.008317,...,0.007930,0.008464,0.008578,0.008065,0.008156,0.008256,0.008681,0.008153,0.007782,1.020251
2010-01-05,0.010497,0.008541,0.007843,0.008238,0.007458,0.008266,0.007962,0.007825,0.007866,0.007981,...,0.009073,0.008563,0.008578,0.007975,0.008369,0.009486,0.008965,0.008494,0.007570,1.038765
2010-01-06,0.012033,0.008065,0.008065,0.007978,0.007458,0.008670,0.009085,0.008464,0.007387,0.008065,...,0.008938,0.008263,0.008871,0.007885,0.008825,0.008700,0.008776,0.008252,0.007570,1.049495
2010-01-07,0.011777,0.008065,0.007917,0.007978,0.006367,0.009878,0.008779,0.008224,0.007412,0.007812,...,0.009409,0.008464,0.008138,0.007885,0.010134,0.009546,0.008824,0.008405,0.007330,1.055437
2010-01-08,0.012673,0.007595,0.008065,0.008325,0.006185,0.009676,0.008677,0.008623,0.008019,0.007476,...,0.008938,0.008761,0.008358,0.007975,0.009708,0.008234,0.009013,0.008788,0.007238,1.054781
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2010-12-24,0.000000,0.000000,0.000000,0.004665,0.000000,0.001808,0.000000,0.000000,0.000000,0.000000,...,0.006478,0.002568,0.003043,0.000822,0.000000,0.004309,0.007344,0.006663,0.007874,0.801721
2010-12-27,0.000000,0.000000,0.000000,0.004509,0.000000,0.001726,0.000000,0.000000,0.000000,0.000000,...,0.006384,0.002528,0.003066,0.000730,0.000000,0.004342,0.007495,0.006553,0.007760,0.801872
2010-12-28,0.000000,0.000000,0.000000,0.005186,0.000000,0.001753,0.000000,0.000000,0.000000,0.000000,...,0.006384,0.002410,0.003066,0.000731,0.000000,0.004320,0.007002,0.006497,0.007586,0.790955


In [7]:
import pyfolio as pf
import FinanceDataReader as fdr

In [8]:
df = fdr.DataReader('US500')
return_series = df['Close'].pct_change().fillna(0)

In [12]:
df['Close'].pct_change()

Date
2006-01-04         NaN
2006-01-05    0.000016
2006-01-06    0.009399
2006-01-09    0.003656
2006-01-10   -0.000357
                ...   
2020-12-16    0.001773
2020-12-17    0.005758
2020-12-18   -0.003511
2020-12-21   -0.003906
2020-12-22   -0.002073
Name: Close, Length: 3768, dtype: float64

In [9]:
pf.create_returns_tear_sheet(return_series)

Start date,2006-01-04
End date,2020-12-22
Total months,179
,Backtest
Annual return,7.4%
Cumulative returns,189.5%
Annual volatility,20.3%
Sharpe ratio,0.45
Calmar ratio,0.13
Stability,0.79
Max drawdown,-56.8%


AttributeError: 'numpy.int64' object has no attribute 'to_pydatetime'

In [251]:
# test_list=[]

# test = yield_change(stock_list_all[0],bgn,end)
# test
# test_list.append(test['Change_all'][-1])

stock_list = []
for i in stock_list_all[0]:
    if str('%06d'%(int(i))) in change.columns:
        stock_list.append(str('%06d'%(int(i))))
    else:
        print('%06d'%(int(i)))

dist_list = []
for j in range(len(stock_list)):
    dist_list.append(1/len(stock_list))

date1 = date_str(bgn)
date2 = date_str(end)

# 수익률 dataframe으로 만드는 함수 추가
df_yield = pd.DataFrame()
for i in range(len(stock_list)):
    stock_code = '%06d'%(int(stock_list[i]))
    try:
        df_temp = change.loc[date1:date2][stock_code].to_frame()
    except:
        print(stock_code)
    df_temp.columns = ['Change']
    df_temp['Change_acc']=(1+df_temp['Change']).cumprod() * dist_list[i]
    df_temp = df_temp.drop(['Change'],axis=1)
    df_temp = df_temp.rename(columns={'Change_acc':stock_code})
    df_yield = pd.concat([df_yield,df_temp],axis=1)
df_yield.loc[date1:date2,'Change_all']=df_yield.sum(axis=1)
df_yield

,045400,064060,001600,033280,054020,054150,004820,038690,037110,024870,...,003310,053040,053700,054010,015170,035200,060240,015260,036620,Change_all
Date,,,,,,,,,,,,,,,,,,,,,
2010-01-04,0.009217,0.007595,0.007917,0.008151,0.007398,0.008468,0.007656,0.007825,0.008141,0.008317,...,0.007930,0.008464,0.008578,0.008065,0.008156,0.008256,0.008681,0.008153,0.007782,1.020251
2010-01-05,0.010497,0.008541,0.007843,0.008238,0.007458,0.008266,0.007962,0.007825,0.007866,0.007981,...,0.009073,0.008563,0.008578,0.007975,0.008369,0.009486,0.008965,0.008494,0.007570,1.038765
2010-01-06,0.012033,0.008065,0.008065,0.007978,0.007458,0.008670,0.009085,0.008464,0.007387,0.008065,...,0.008938,0.008263,0.008871,0.007885,0.008825,0.008700,0.008776,0.008252,0.007570,1.049495
2010-01-07,0.011777,0.008065,0.007917,0.007978,0.006367,0.009878,0.008779,0.008224,0.007412,0.007812,...,0.009409,0.008464,0.008138,0.007885,0.010134,0.009546,0.008824,0.008405,0.007330,1.055437
2010-01-08,0.012673,0.007595,0.008065,0.008325,0.006185,0.009676,0.008677,0.008623,0.008019,0.007476,...,0.008938,0.008761,0.008358,0.007975,0.009708,0.008234,0.009013,0.008788,0.007238,1.054781
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2010-12-24,0.000000,0.000000,0.000000,0.004665,0.000000,0.001808,0.000000,0.000000,0.000000,0.000000,...,0.006478,0.002568,0.003043,0.000822,0.000000,0.004309,0.007344,0.006663,0.007874,0.801721
2010-12-27,0.000000,0.000000,0.000000,0.004509,0.000000,0.001726,0.000000,0.000000,0.000000,0.000000,...,0.006384,0.002528,0.003066,0.000730,0.000000,0.004342,0.007495,0.006553,0.007760,0.801872
2010-12-28,0.000000,0.000000,0.000000,0.005186,0.000000,0.001753,0.000000,0.000000,0.000000,0.000000,...,0.006384,0.002410,0.003066,0.000731,0.000000,0.004320,0.007002,0.006497,0.007586,0.790955


In [257]:
test_list_all=[]
start = time.time()
for k in range(2008,2020):
#     기간
    bgn=datetime.datetime(int(k),1,1)
    end=datetime.datetime(int(k)+1,1,1)
    # 조건 (시가총액으로 종목정렬)
    df_base_year_list = making_cost_list(bgn)
    # 종목 구성
    div = 10 # 몇개로 쪼갤지
    stock_list_all=[]
    for i in range(div):
        stock_list_temp = df_base_year_list[int(len(df_base_year_list)/div*i):int(len(df_base_year_list)/div*(i+1))]
        stock_list_all.append(stock_list_temp)
#     print('step1')
    # 결과 저장
    test_list=[]
    for i in range(len(stock_list_all)):
        test = yield_change(stock_list_all[i],bgn,end)
        test_list.append(test['Change_all'][-1])
#         print(i)
    test_list_all.append(test_list)
    print((time.time()-start)/60)

053250
054370
008030
025690
036020
054080
012250
037220
008120
037640
056060
006750
036590
038960
041500
009890
038980
016970
006150
063510
055810
009940
032050
056500
001580
049130
025830
038420
048760
045310
019560
054250
006600
009760
025930
014900
039870
063350
049370
001830
002860
0.15369592507680258
016970
038420
056060
063510
054250
041500
047420
066480
045310
054440
054790
037060
048760
006600
003040
011020
005330
063350
049370
003330
025930
0.33915642499923704
037060
045310
054250
038420
054790
054440
063350
011020
005330
025930
067910
001830
0.5083554029464722
0.6600589950879415
0.8047472635904948
1.0002150098482767
1.175871694087982
1.3461312214533487
1.5221226414044697
1.6685689727465312
1.826797572771708
2.0150914510091145
2.1922440846761067
2.3720189332962036
2.543301733334859


In [226]:
backtest.cumprod().iloc[-1]

1     2.216979
2     3.132587
3     2.881954
4     1.127112
5     1.245976
6     1.246814
7     1.394879
8     1.146278
9     1.112459
10    1.014702
Name: 2019, dtype: float64

In [264]:
backtest=pd.DataFrame(data=test_list_all,index=range(2005,2020),columns=range(1,11))
# list(range(1,11))
backtest

,1,2,3,4,5,6,7,8,9,10
2005,3.545446,2.418225,2.137422,1.947922,2.024609,2.008392,2.078975,1.828364,1.923480,1.675027
2006,1.187925,0.927088,0.915339,0.882807,0.905370,0.810726,0.845287,0.875961,0.983376,1.112283
2007,1.369854,1.113020,1.204904,1.033963,1.138921,1.150698,1.185035,1.201012,1.355597,1.510532
2008,0.407996,0.431708,0.431897,0.399756,0.458022,0.537223,0.489723,0.524101,0.537738,0.558431
2009,0.866063,1.329426,1.646736,1.618130,1.590449,1.689918,1.455773,1.456742,1.472940,1.506184
2010,0.801266,1.001622,1.508246,0.958617,0.960708,1.078798,1.072714,1.029360,1.116561,1.268338
2011,0.827984,1.055966,0.901418,0.899460,1.008297,1.044975,0.993014,0.979920,0.877977,0.854633
2012,1.001803,1.156158,1.120547,1.120447,1.033745,0.961284,1.060052,0.982144,0.998672,1.054018
2013,1.049168,0.943511,1.097148,1.006657,0.999674,1.047892,1.025587,1.037086,1.009062,1.018982
2014,1.040962,1.157248,1.153676,1.122760,1.157957,1.128169,1.224114,1.097605,1.082064,0.941095


In [266]:
backtest.iloc[3:].cumprod()

,1,2,3,4,5,6,7,8,9,10
2008,0.407996,0.431708,0.431897,0.399756,0.458022,0.537223,0.489723,0.524101,0.537738,0.558431
2009,0.353350,0.573924,0.711220,0.646857,0.728460,0.907863,0.712925,0.763479,0.792055,0.841100
2010,0.283128,0.574855,1.072694,0.620088,0.699837,0.979401,0.764764,0.785894,0.884378,1.066798
2011,0.234425,0.607027,0.966946,0.557744,0.705644,1.023449,0.759421,0.770113,0.776464,0.911721
2012,0.234848,0.701819,1.083508,0.624923,0.729456,0.983825,0.805026,0.756362,0.775432,0.960971
2013,0.246395,0.662174,1.188768,0.629083,0.729218,1.030943,0.825624,0.784412,0.782459,0.979212
2014,0.256488,0.766299,1.371454,0.706309,0.844404,1.163078,1.010658,0.860975,0.846671,0.921531
2015,0.467502,1.270381,2.031536,0.861887,1.070594,1.398752,1.152276,1.046223,1.038147,0.967917
2016,0.651861,1.710542,2.301006,0.921774,1.019767,1.350736,1.023642,0.961541,0.917590,0.895538
2017,0.633248,1.692941,2.087156,0.867955,0.963337,1.321594,1.092591,1.056956,1.139093,1.039912


### PBR Backtest

In [52]:
test_list_all=[]
start = time.time()
for k in ['2015','2016','2017','2018','2019']:
    # 기간
    bgn=datetime.datetime(int(k)+1,3,1)
    end=datetime.datetime(int(k)+2,3,1)
    # 조건
    df_base_year = df_base.dropna(subset=['PBR_{}'.format(k)]) #결측치 제외
    df_base_year = df_base_year[df_base_year['PBR_{}'.format(k)] > 0.2] #0.2보다 작은 경우 제외(BPS가 마이너스)
    df_base_year_list = df_base_year.sort_values(by='PBR_{}'.format(k), ascending=True).index
    # 종목 구성
    div = 100 # 몇개로 쪼갤지
#     stock_list_all=[]
#     for i in range(div):
#         stock_list_temp = df_base_year_list[int(len(df_base_year_list)/div*i):int(len(df_base_year_list)/div*(i+1))]
#         stock_list_all.append(stock_list_temp)
    stock_list_all = [df_base_year_list[0:20]]
    # 결과 저장
    test_list=[]
    for i in range(len(stock_list_all)):
        test = yield_fdr(stock_list_all[i],bgn,end)
        test_list.append(test['Change_all'][-1])
    test_list_all.append(test_list)
print((time.time()-start)/60)
    # 그림 만들기
#     plt.figure()
#     plt.plot(list(range(len(test_list))),test_list)
#     plt.show()

0.4106587847073873


In [41]:
start = 1
for j in test_list_all:
    start = start * float(j[0])
    print(start)

1.2530244095450587
1.2757366870223505
1.2228293992621235
1.0414384619162518
1.4127464898096909


In [59]:
df_yield.loc[df_yield.index[0],'Change'] = df_yield.loc[df_yield.index[0],'Change_all']
for i in range(len(df_yield.index)-1):
    df_yield.loc[df_yield.index[i+1],'Change'] = df_yield.loc[df_yield.index[i+1],'Change_all'] / df_yield.loc[df_yield.index[i],'Change_all']
df_yield

,003830,000950,006370,003300,003200,016610,001430,002820,040610,000860,...,002920,900260,067830,028080,015750,001620,001230,Change_all,Drawdown,Change
Date,,,,,,,,,,,,,,,,,,,,,
2020-03-02,0.049878,0.048654,0.049190,0.051632,0.050488,0.049506,0.050000,0.048618,0.050984,0.049155,...,0.050211,0.049880,0.047637,0.050743,0.051645,0.050324,0.049412,1.000210,0.000000,1.000210
2020-03-03,0.049332,0.050000,0.049352,0.053264,0.050488,0.050000,0.050459,0.048289,0.050656,0.049437,...,0.050106,0.049398,0.047259,0.050330,0.051645,0.050648,0.048665,1.001846,0.000000,1.001636
2020-03-04,0.049696,0.050769,0.050162,0.053116,0.049919,0.050494,0.050000,0.046776,0.052623,0.050000,...,0.052537,0.051205,0.047826,0.050330,0.052138,0.050810,0.049412,1.013737,0.000000,1.011869
2020-03-05,0.050000,0.051154,0.052188,0.052077,0.049919,0.051695,0.050459,0.046974,0.053115,0.050423,...,0.054757,0.052530,0.048204,0.050990,0.052549,0.052350,0.050053,1.027642,0.000000,1.013716
2020-03-06,0.049575,0.049615,0.051540,0.053042,0.049431,0.050706,0.048624,0.046579,0.051803,0.050000,...,0.053066,0.051928,0.046881,0.050495,0.050822,0.051053,0.047650,1.007225,1.986769,0.980132
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-11-16,0.056501,0.114423,0.075851,0.073591,0.060163,0.061864,0.047018,0.059474,0.056721,0.049718,...,0.060994,0.042169,0.055198,0.072360,0.063816,0.050081,0.076068,1.329466,0.736113,1.003380
2020-11-17,0.057290,0.114808,0.078120,0.077151,0.060407,0.063206,0.047706,0.058421,0.054590,0.050986,...,0.061099,0.040964,0.055671,0.070297,0.068421,0.050972,0.075427,1.343669,0.000000,1.010684
2020-11-18,0.061968,0.121154,0.081442,0.078042,0.061301,0.063630,0.047477,0.058618,0.054918,0.050986,...,0.060888,0.041807,0.056238,0.069802,0.066776,0.051459,0.074145,1.352982,0.000000,1.006931


In [65]:
df_yield=test
btw=((end-bgn).days/365)
# CAGR
CR = df_yield['Change_all'][-1]
print('누적수익률 [%] = {0:0.2f}'.format((CR-1)*100))
CAGR = (df_yield['Change_all'][-1])**(1/btw)
print('CAGR [%] = {0:0.2f}'.format((CAGR-1)*100))
# MDD 계산
df_yield['Drawdown']=(df_yield['Change_all'].cummax()-df_yield['Change_all'])/df_yield['Change_all'].cummax()*100
MDD = df_yield['Drawdown'].max()
# print(type(MDD))
# print('MDDtest = {0.2f}'.format(1/3))
print('MDD = {0:0.2f}'.format(MDD))
# 누적 수익
initial_money = 10**4
cummulative_return = initial_money*df_yield['Change_all'][-1]-initial_money
print('누적수익 [\] = {0:0.2f}'.format(cummulative_return))
df_yield.loc[df_yield.index[0],'Change'] = df_yield.loc[df_yield.index[0],'Change_all']
for i in range(len(df_yield.index)-1):
    df_yield.loc[df_yield.index[i+1],'Change'] = df_yield.loc[df_yield.index[i+1],'Change_all'] / df_yield.loc[df_yield.index[i],'Change_all']
print('분산 [%] = {0:0.2f}'.format(df_yield['Change'].std()*100))
# 샤프지수 계산 (risk free rate=0)
sharpe_ratio = (CAGR-1)/(df_yield['Change'].std()*(252**0.5))
# mean = df_yield['Change_all'][-1]**(1/len(df_yield.index))
# sharpe_ratio = mean/df_yield['Change'].std()
# A_sharpe_ratio = (252**0.5)*sharpe_ratio
print('Sharpe_Ratio = {0:0.2f}'.format(sharpe_ratio))
# print('Annual_Sharpe_Ratio [%] = {0:0.2f}'.format(A_sharpe_ratio*100))

누적수익률 [%] = 35.65
CAGR [%] = 35.65
MDD = 37.17
누적수익 [\] = 3565.34
분산 [%] = 2.27
Sharpe_Ratio = 0.99


In [64]:
(df_yield['Change'].std()*(252**0.5))

0.3607508743358401

# 나중에 정리할 팁들

In [28]:
# marcap은 그 당시 실제 주가를 반영
df_marcap = marcap_data('2016-03-02')
df_marcap[df_marcap['Code'] == '005930']
# fdr은 액면분할한 경우 변환하여 당시 주가를 계산 (fnguide는 이 EPS, BPS를 반영함.)
fdr.DataReader('005930','2016-3-1')

,Code,Name,Close,Changes,ChagesRatio,Volume,Amount,Open,High,Low,Marcap,MarcapRatio,Stocks,ForeignShares,ForeignRatio,Rank
Date,,,,,,,,,,,,,,,,
2016-03-02,005930,삼성전자,1197000.0,19000.0,1.6,236372.0,2.838530e+11,1200000.0,1207000.0,1196000.0,1.736480e+14,12.11,145069337.0,71874263.0,49.54,1.0


In [48]:
# df_temp_temp = df_temp.iloc[:,0].to_frame()
# df_temp_temp.drop(axis=0)
# 결측치인 경우만 보기
df_temp[df_temp['BPS(원)_2018'].isnull()]
# 결측치인 경우 제외하기
df_temp.dropna(subset=['BPS(원)_2018'])

,BPS(원)_2018,시가총액_2018
095570,6131.0,b
006840,64235.0,NaN
027410,15722.0,NaN
282330,30282.0,NaN
138930,23401.0,NaN
...,...,...
024060,4742.0,NaN
010240,5004.0,NaN
189980,1899.0,NaN
037440,4304.0,NaN


In [ ]:
bgn=datetime.datetime(2018,1,1)
date1 = date_str(bgn)
date2 = date_str(bgn+relativedelta(days=7))
first_date=fdr.DataReader('005930',date1,date2).index[0] #삼성전자보다 먼저 상장된 종목은 검토 불가(1996-12-05)
df_all = marcap_data(first_date).loc[:,['Code','Marcap']]
# df_all
# stock_list = marcap_data(first_date)['Code'].to_list() # 이 list는 현재 상장종목 list인듯

def marcap_df (bgn):
    date1 = date_str(bgn)
    date2 = date_str(bgn+relativedelta(days=7))
    first_date=fdr.DataReader('005930',date1,date2).index[0]
    df_all = marcap_data(first_date).loc[:,['Code','Marcap']]
    df_all.index = df_all['Code']
    df_all.drop(columns = ['Code'],inplace=True)
    return df_all